In [18]:
import torch
import ffmpeg
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 10)

## Download YOLO

In [1]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True)

Using cache found in /Users/dmevdok/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      7040  models.common.Focus                     [3, 64, 3]                    
  1                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  2                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  3                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  4                -1  1   1611264  models.common.C3                        [256, 256, 9]                 
  5                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  6                -1  1   6433792  models.common.C3                        [512, 512, 9]                 
  7                -1  1   4720640  models.common.Conv          

Adding autoShape... 


## YOLO inference on frames

In [26]:
WIDTH = 640

probe = ffmpeg.probe('../data/phase_I/video_0.mp4')
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
old_width = int(video_info['width'])
old_height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

new_width = WIDTH
new_height = int(old_height / (old_width / new_width))

out, err = (
    ffmpeg
    .input('../data/phase_I/video_0.mp4')
    .filter('scale', new_width, new_height)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)
loaded_video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, new_height, new_width, 3])
)

def show_bboxes(im, bboxes_tensor):
    fig, ax = plt.subplots()
    ax.imshow(im)
    for bbox in bboxes_tensor:
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        

@interact(frame=(0, num_frames-1))
def show_frame(frame=0):
    prediction = model([loaded_video[frame,:,:,:]]).xyxy[0]
    predicted_traffic_lights = prediction[prediction[:,5] == 9]    
    show_bboxes(loaded_video[frame,:,:,:], predicted_traffic_lights)

interactive(children=(IntSlider(value=0, description='frame', max=1202), Output()), _dom_classes=('widget-inte…

## YOLO preforms rather good